# Ensemble model - AN2DL - Challenge 1

### Libraries and mounting

In [ ]:
import gc
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
import random
import time
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras as tfk
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Average
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import mixed_precision

input_shape = (96,96,3)

In [ ]:
# Clear out any old model state.
gc.collect()
tfk.backend.clear_session()

In [ ]:
print("TensorFlow:", tf.__version__)

TensorFlow: 2.12.0


In [ ]:
'''# Code to run the training on the TPU taken from
# https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/shakespeare_with_tpu_and_keras.ipynb#scrollTo=ExQ922tfzSGA
tf.keras.backend.clear_session()

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)'''

All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/UNI/MAGISTRALE/ANNDL/Challenge 1/ensemble_trials'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/My Drive/UNI/MAGISTRALE/ANNDL/Challenge 1/ensemble_trials'
/content/drive/.shortcut-targets-by-id/17vuJrJgDzlV1IYNFiwDvQFTgjdlP3Xnd/Challenge 1/ensemble_trials


### Dataset generator (for large training dataset)
A larger augmented, balanced dataset of 40k samples for training and 5k for validation. It is loaded in ram in batches to avoid out of memory errors

In [ ]:
def load_augmented_batches(batch_dir, preprocess_fn, batch_size):
    batch_files = [os.path.join(batch_dir, file_name) for file_name in os.listdir(batch_dir) if file_name.endswith('.npz')]
    while True:  # Loop indefinitely
        np.random.shuffle(batch_files)  # Shuffle the order of files to introduce randomness
        for batch_file in batch_files:
            # Load the entire batch file
            batch_data = np.load(batch_file)
            images = batch_data['data']
            labels = batch_data['labels']

            # Shuffle the data
            indices = np.arange(len(images))
            np.random.shuffle(indices)
            images = images[indices]
            labels = labels[indices]

            # Yield batches from the current file
            for i in range(0, len(images), batch_size):
                end_index = i + batch_size if (i + batch_size) < len(images) else len(images)
                batch_images = images[i:end_index]
                batch_labels = labels[i:end_index]
                gc.collect()
                #yield preprocess_fn(batch_images), batch_labels
                yield batch_images, batch_labels

In [ ]:
#Fede's augmented data path
augmented_data_path_train = "../Data/Augmented_Experimental/Train"
augmented_data_path_val = "../Data/Augmented_Experimental/Validation"

npz_file_size = 5000
batch_size = 64

train_generator = load_augmented_batches(augmented_data_path_train, None, batch_size)
val_generator = load_augmented_batches(augmented_data_path_val, None, batch_size)

# Determine the number of steps per epoch (number of batches)
train_steps = sum(1 for _ in os.listdir(augmented_data_path_train) if _.endswith('.npz')) * (npz_file_size // batch_size)
val_steps = sum(1 for _ in os.listdir(augmented_data_path_val) if _.endswith('.npz')) * (npz_file_size // batch_size)

### Full dataset loading (for smaller training datasets)
An augmented dataset of 11500 samples for training and around 1000 for validation, it fits the RAM costraints of colab. Classes did not get balanced during the augmentation

In [ ]:
# Load data
train = np.load('../Data/Augmented_Unbalanced/Train/11.5k.npz', allow_pickle=True)
X_train, y_train = train["data"], train["labels"]

val = np.load('../Data/Augmented_Unbalanced/Validation/valid_exp_split.npz', allow_pickle=True)
X_val, y_val = val["data"], val["labels"]

We tried using sklearn compute_class_weight ```compute_class_weight``` to balance the 2 classes, but we did not get any significant improvement.



In [ ]:
class_weights = compute_class_weight(
    'balanced',
    classes = np.unique(y_train),
    y = y_train
)

# Conversion needed for fit function
class_weight_dict = dict(enumerate(class_weights))

## Single feature extractors


### ConvNeXtLarge

In [ ]:
with strategy.scope():
  convnext = tfk.applications.ConvNeXtLarge(
    input_shape = input_shape,
    include_top = False,
    pooling='avg'
  )
'''  convnext.trainable = True
  for layer in convnext.layers[:-5]:  # Replace n with the number of layers you want to freeze
      layer.trainable = False'''


785596384/785596384 [==============================] - 8s 0us/step


'  convnext.trainable = True\n  for layer in convnext.layers[:-5]:  # Replace n with the number of layers you want to freeze\n      layer.trainable = False'

In [ ]:
convnext.trainable = False

In [ ]:
convnext.summary()

Model: "convnext_large"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 convnext_large_prestem_normali  (None, 96, 96, 3)   0           ['input_1[0][0]']                
 zation (Normalization)                                                                           
                                                                                                  
 convnext_large_stem (Sequentia  (None, 24, 24, 192)  9792       ['convnext_large_prestem_normaliz
 l)                                                              ation[0][0]']                    
                                                                                     

### MobileNetV2

In [ ]:
with strategy.scope():
  mobilenet = tfk.applications.MobileNetV2(
    input_shape = input_shape,
    include_top = False,
    pooling='avg'
  )
'''  mobilenet.trainable = True
  for layer in mobilenet.layers[:-3]:  # Replace n with the number of layers you want to freeze
      layer.trainable = False'''

9406464/9406464 [==============================] - 0s 0us/step


'  mobilenet.trainable = True\n  for layer in mobilenet.layers[:-3]:  # Replace n with the number of layers you want to freeze\n      layer.trainable = False'

In [ ]:
mobilenet.trainable = False

In [ ]:
mobilenet.summary()

Model: "mobilenetv2_1.00_96"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 48, 48, 32)   864         ['input_2[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 48, 48, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 48, 48, 32)   0           ['bn_Conv1[0][0]']               
                                                                                

### EfficientNetV2L

In [ ]:
with strategy.scope():
  efficientnet = tfk.applications.EfficientNetV2L(
    input_shape = input_shape,
    include_top = False,
    pooling='avg'
  )
'''  efficientnet.trainable = True
  for layer in efficientnet.layers[:-3]:  # Replace n with the number of layers you want to freeze
      layer.trainable = False'''

473176280/473176280 [==============================] - 3s 0us/step


'  efficientnet.trainable = True\n  for layer in efficientnet.layers[:-3]:  # Replace n with the number of layers you want to freeze\n      layer.trainable = False'

In [ ]:
efficientnet.trainable = False

In [ ]:
efficientnet.summary()

Model: "efficientnetv2-l"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 96, 96, 3)    0           ['input_3[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 48, 48, 32)   864         ['rescaling[0][0]']              
                                                                                                  
 stem_bn (BatchNormalization)   (None, 48, 48, 32)   128         ['stem_conv[0][0]']              
                                                                                   

### Xception

In [ ]:
with strategy.scope():
  xception = tfk.applications.Xception(
    input_shape = input_shape,
    include_top = False,
    pooling='avg'
  )
'''  xception.trainable = True
  for layer in xception.layers[:-3]:  # Replace n with the number of layers you want to freeze
      layer.trainable = False'''

83683744/83683744 [==============================] - 1s 0us/step


'  xception.trainable = True\n  for layer in xception.layers[:-3]:  # Replace n with the number of layers you want to freeze\n      layer.trainable = False'

In [ ]:
xception.trainable = False

In [ ]:
xception.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 47, 47, 32)   864         ['input_4[0][0]']                
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 47, 47, 32)  128         ['block1_conv1[0][0]']           
 ation)                                                                                           
                                                                                                  
 block1_conv1_act (Activation)  (None, 47, 47, 32)   0           ['block1_conv1_bn[0][0]'] 

## Single models
creation of single different models, each will be based on the feature extraction part of one of the keras.applications models, plus one or more dense layers (Relu), followed by the usual single node output layer (sigmoid)

In [ ]:
dense_units = [512,512,512]
#with strategy.scope()
x = tfk.layers.Dense(dense_units[0], name="convnext_dense", kernel_regularizer=tfk.regularizers.l2(1e-4),activation='relu')(convnext.output)
x = tfk.layers.BatchNormalization()(x)
x = tfk.layers.Dense(dense_units[1], name="convnext_dense_2", kernel_regularizer=tfk.regularizers.l2(1e-4),activation='relu')(x)
x = tfk.layers.BatchNormalization()(x)
x = tfk.layers.Dense(dense_units[2], name="convnext_dense_3", kernel_regularizer=tfk.regularizers.l2(1e-4),activation='relu')(x)
x = tfk.layers.BatchNormalization()(x)
prediction = tfk.layers.Dense(1, activation='sigmoid', kernel_regularizer=tfk.regularizers.l2(1e-4), name="convnext_output")(x)
convnext_model = Model(inputs=convnext.inputs, outputs=prediction)

x = tfk.layers.Dense(dense_units[0], name="mobilenet_dense", kernel_regularizer=tfk.regularizers.l2(1e-4),activation='relu')(mobilenet.output)
x = tfk.layers.BatchNormalization()(x)
x = tfk.layers.Dense(dense_units[1], name="mobilenet_dense_2", kernel_regularizer=tfk.regularizers.l2(1e-4),activation='relu')(x)
x = tfk.layers.BatchNormalization()(x)
x = tfk.layers.Dense(dense_units[2], name="mobilenet_dense_3", kernel_regularizer=tfk.regularizers.l2(1e-4),activation='relu')(x)
x = tfk.layers.BatchNormalization()(x)
prediction = tfk.layers.Dense(1, activation='sigmoid',kernel_regularizer=tfk.regularizers.l2(1e-4), name="mobilenet_output")(x)
mobilenet_model = Model(inputs=mobilenet.inputs, outputs=prediction)

x = tfk.layers.Dense(dense_units[0], name="efficientnet_dense", kernel_regularizer=tfk.regularizers.l2(1e-4),activation='relu')(efficientnet.output)
x = tfk.layers.BatchNormalization()(x)
x = tfk.layers.Dense(dense_units[1], name="efficientnet_dense_2", kernel_regularizer=tfk.regularizers.l2(1e-4),activation='relu')(x)
x = tfk.layers.BatchNormalization()(x)
x = tfk.layers.Dense(dense_units[2], name="efficientnet_dense_3", kernel_regularizer=tfk.regularizers.l2(1e-4),activation='relu')(x)
x = tfk.layers.BatchNormalization()(x)
prediction = tfk.layers.Dense(1, activation='sigmoid',kernel_regularizer=tfk.regularizers.l2(1e-4), name="efficientnet_output")(x)
efficientnet_model = Model(inputs=efficientnet.inputs, outputs=prediction)

x = tfk.layers.Dense(dense_units[0], name="xception_dense", kernel_regularizer=tfk.regularizers.l2(1e-4),activation='relu')(xception.output)
x = tfk.layers.BatchNormalization()(x)
x = tfk.layers.Dense(dense_units[1], name="xception_dense_2", kernel_regularizer=tfk.regularizers.l2(1e-4),activation='relu')(x)
x = tfk.layers.BatchNormalization()(x)
x = tfk.layers.Dense(dense_units[2], name="xception_dense_3", kernel_regularizer=tfk.regularizers.l2(1e-4),activation='relu')(x)
x = tfk.layers.BatchNormalization()(x)
prediction = tfk.layers.Dense(1, activation='sigmoid' ,kernel_regularizer=tfk.regularizers.l2(1e-4),name="xception_output")(x)
xception_model = Model(inputs=xception.inputs, outputs=prediction)

## Ensembling all single models into one

In [ ]:
#with strategy.scope():
single_models = [convnext_model, mobilenet_model, efficientnet_model, xception_model]
inputs = tfk.Input(shape=input_shape)
model_outputs = [model(inputs) for model in single_models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=inputs, outputs=ensemble_output, name='ensemble')
# Use custom lr scheduler
lr_schedule = tfk.optimizers.schedules.ExponentialDecay(
  initial_learning_rate=1e-3,  # Start with this learning rate
  decay_steps=900,             # After how many steps to apply decay (More or less 11k dataset has 370 steps per epoch, so after about 3 epochs start decreasing lr)
  decay_rate=0.9,              # Decay rate
  staircase=True               # If True, learning rate is reduced at discrete intervals
)

optimizer = tfk.optimizers.AdamW(learning_rate = lr_schedule)
ensemble_model.compile(
    loss=tfk.losses.BinaryCrossentropy(),
    optimizer=optimizer,
    metrics=['accuracy']
)
ensemble_model.summary()

#callbacks
reduce_lr_on_plateau = tfk.callbacks.ReduceLROnPlateau(
  monitor='val_loss',
  factor=0.9,
  patience=5
)
es = tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=30, restore_best_weights=True)

Model: "ensemble"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 model_4 (Functional)           (None, 1)            197017793   ['input_6[0][0]']                
                                                                                                  
 model_5 (Functional)           (None, 1)            2914369     ['input_6[0][0]']                
                                                                                                  
 model_6 (Functional)           (None, 1)            118403233   ['input_6[0][0]']                
                                                                                           

## Training

### Train with data generator

In [ ]:
gc.collect()
#with strategy.scope():
history = ensemble_model.fit(
    x = train_generator,  # Training data comes from the generator
    steps_per_epoch = train_steps,  # Number of batches in the training set
    epochs = 1000,
    validation_data = val_generator,  # Validation data comes from the generator
    validation_steps = val_steps,  # Number of batches in the validation set
    callbacks = [es, reduce_lr_on_plateau]
).history

### Train on full data

In [ ]:
gc.collect()
with strategy.scope():
  history = ensemble_model.fit(
    X_train,
    y_train,
    batch_size = 64,
    #class_weight = class_weight_dict,
    validation_data=(X_val, y_val),
    epochs = 1000,
    callbacks = [es]
  ).history

Epoch 1/1000
188/188 [==============================] - 294s 849ms/step - loss: 0.8326 - accuracy: 0.6199 - val_loss: 0.7700 - val_accuracy: 0.6144
Epoch 2/1000
188/188 [==============================] - 19s 103ms/step - loss: 0.7425 - accuracy: 0.6210 - val_loss: 0.7357 - val_accuracy: 0.6144
Epoch 3/1000
188/188 [==============================] - 19s 103ms/step - loss: 0.7204 - accuracy: 0.6210 - val_loss: 0.6917 - val_accuracy: 0.6144
Epoch 4/1000
188/188 [==============================] - 35s 187ms/step - loss: 0.6034 - accuracy: 0.6259 - val_loss: 0.5820 - val_accuracy: 0.6164
Epoch 5/1000
188/188 [==============================] - 34s 182ms/step - loss: 0.5759 - accuracy: 0.6477 - val_loss: 0.5638 - val_accuracy: 0.7133
Epoch 6/1000
188/188 [==============================] - 33s 177ms/step - loss: 0.5656 - accuracy: 0.7026 - val_loss: 0.5369 - val_accuracy: 0.7323
Epoch 7/1000
188/188 [==============================] - 33s 178ms/step - loss: 0.5102 - accuracy: 0.8154 - val_loss: 

## Plot history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_loss'], label='ensemble model', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_accuracy'], label='ensemble model', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

dense12 no lr schedule, no rl reduce, no kernel reg: <86 \\
dense12 no lr schedule, no rl reduce, yes kernel reg: 87.21 \\
dense12 yes lr schedule, no rl reduce, yes kernel reg: 87.41 \\
dense16 yes lr schedule, no rl reduce, yes kernel reg: 88.11 \\
same but patience 30: 88.91

## Save (and reload)

In [ ]:
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
ensemble_model.save('ensemble_model_v2_DENSE512', options=save_locally)
#ensemble_model.save('ensemble_model_v2_DENSE512')

In [ ]:
del ensemble_model

In [ ]:
ensemble_model = tfk.models.load_model('ensemble_model_dense512')

In [ ]:
test_dir = '../Data/Augmented/Test'
test_generator = load_augmented_batches(train_dir, preprocess_input, batch_size)
test_steps = sum(1 for _ in os.listdir(test_dir) if _.endswith('.npz'))

In [ ]:
model.evaluate(
    x=test_generator,
    steps=test_steps
)